In [1]:
import pandas as pd, numpy as np, json, itertools, bangla
import datetime as dt
import pandas as pd
m_final = pd.read_csv("#################.csv")
m_final.columns

C:\Users\tanvi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Index(['Unnamed: 0', 'coordinator_id', 'household_id', 'user_id', 'member_id',
       'age', 'gender', 'diabetic', 'profile_hypertensive', 'measurement_id',
       'attr_id', 'attr_code', 'mes_tag_id', 'SYSTOLIC', 'TIME-SYS',
       'DIASTOLIC', 'TIME-DIA', 'RESULT_STAT_BP', 'BMI', 'HEIGHT', 'TIME-HI',
       'WEIGHT', 'TIME-WI', 'RESULT_STAT_BMI', 'SUGAR', 'TAG_NAME', 'TIME-SU',
       'RESULT_STAT_SUGAR', 'PULSE_RATE', 'TIME-PR', 'RESULT_STAT_PR', 'SPO2',
       'TIME-SPO2', 'RESULT_STAT_SPO2', 'MUAC', 'TIME-MUAC',
       'RESULT_STAT_MUAC', 'Unnamed: 3'],
      dtype='object')

In [ ]:
import pandas as pd, numpy as np, json, itertools, bangla
import datetime as dt
import pandas as pd
m_final = pd.read_csv("##########.csv")
m_final.columns

In [2]:
m_final.shape

(145734, 38)

# Overall table

In [3]:
overall_table = pd.DataFrame(columns=['মোট পরিমাপকৃত খানা সংখ্যা', 'মোট পরিমাপকৃত সদস্য সংখ্যা', 'মোট রক্তচাপ পরিমাপ', 'মোট ডায়াবেটিস পরিমাপ','মোট বিএমআই পরিমাপ', 'মোট পালস রেট পরিমাপ', 'মোট এসপিও২ পরিমাপ', 'মোট মুয়াক পরিমাপ'])
overall_table.index.name = 'ইউনিয়ন'

overall_data = [m_final.household_id.nunique(), m_final.user_id.nunique(), m_final[m_final.SYSTOLIC.isnull() == False].user_id.nunique(), m_final[m_final.SUGAR.isnull() == False].user_id.nunique(),m_final[m_final.BMI.isnull() == False].user_id.nunique(), m_final[m_final.PULSE_RATE.isnull() == False].user_id.nunique(), m_final[m_final.SPO2 > 0].user_id.nunique(), m_final[m_final.MUAC.isnull() == False].user_id.nunique()]

overall_data = [bangla.convert_english_digit_to_bangla_digit(v) for v in overall_data]

overall_table.loc['সব'] = overall_data

overall_table

,মোট পরিমাপকৃত খানা সংখ্যা,মোট পরিমাপকৃত সদস্য সংখ্যা,মোট রক্তচাপ পরিমাপ,মোট ডায়াবেটিস পরিমাপ,মোট বিএমআই পরিমাপ,মোট পালস রেট পরিমাপ,মোট এসপিও২ পরিমাপ,মোট মুয়াক পরিমাপ
ইউনিয়ন,,,,,,,,
সব,৮৯৫০৬,১৪৫৭৩৪,১১৬৯৪৯,৭২০৮,২৪৯৩০,১১৬৮৪১,৩৫৩১৯,৩৬১৪


In [4]:
BP_col_before = ['Low', 'Normal', 'Prehypertension', 'Mild High', 'Moderate High', 'Severe High']
BP_col_After = ['Low', 'Normal', 'Mild High', 'Moderate High', 'Severe High']

BMI_col_un18 = ['Underweight', 'Normal', 'Overweight', 'Obesity']
BMI_col_ov18 = ['Underweight', 'Normal', 'Overweight', 'Obesity', 'Highly Obesity', 'Morbid Obesity']

SUGAR_col_before = ['LOW (Hypoglycemia)', 'Normal', 'PRE- DIABETIC', 'DIABETIC (need confirmation)']
SUGAR_col_after = ['LOW (Hypoglycemia)', 'Normal','HIGH (Borderline)','High']

SUGAR_tag = ['Random', 'Fasting', 'OGTT']
PR_col = ['Low', 'Normal', 'High']
MUAC_col = ['Severe Malnutrition', 'Malnutrition', 'Nourished']
SPO2_col = ['Very low', 'Low', 'Normal']

gen_col = ['Male', 'Female']

In [5]:
final = m_final[m_final.age >= 18]

# Blood Pressure Tables

In [6]:
#for blood pressure
bp_table_before = pd.DataFrame(columns=['নিম্ন রক্তচাপ', 'স্বাভাবিক', 'উচ্চ রক্তচাপ (পূর্বাবস্থা)', 'উচ্চ রক্তচাপ (Mild High)', 'উচ্চ রক্তচাপ (Moderate High)', 'মাত্রাতিরিক্ত রক্তচাপ','সর্বমোট'])
bp_table_before.index.name = 'লিঙ্গ'
bp_series_male = []
bp_series_male_percent = []
bp_series_female = []
bp_series_female_percent = []
bp_series_total = []
bp_series_total_percent = []

for state in BP_col_before:
    bp_series_male.append(final[(final.profile_hypertensive==False) & (final.SYSTOLIC.isnull() == False) & (final.gender=='Male') & (final.RESULT_STAT_BP == state)].user_id.nunique())
    bp_series_male_percent.append(((final[(final.profile_hypertensive==False) & (final.SYSTOLIC.isnull() == False) & (final.gender=='Male') & (final.RESULT_STAT_BP == state)].user_id.nunique())/(final[(final.profile_hypertensive==False) &(final.SYSTOLIC.isnull()==False) & (final.RESULT_STAT_BP != 'High')].user_id.nunique()))*100)
    bp_series_female.append(final[(final.profile_hypertensive==False) & (final.SYSTOLIC.isnull() == False) & (final.gender=='Female') & (final.RESULT_STAT_BP == state)].user_id.nunique())
    bp_series_female_percent.append(((final[(final.profile_hypertensive==False) & (final.SYSTOLIC.isnull() == False) & (final.gender=='Female') & (final.RESULT_STAT_BP == state)].user_id.nunique())/(final[(final.profile_hypertensive==False) &(final.SYSTOLIC.isnull()==False) & (final.RESULT_STAT_BP != 'High')].user_id.nunique()))*100)
    bp_series_total.append(final[(final.profile_hypertensive==False) &(final.SYSTOLIC.isnull()==False) & (final.RESULT_STAT_BP == state)].user_id.nunique())
    bp_series_total_percent.append(((final[(final.profile_hypertensive==False) & (final.SYSTOLIC.isnull() == False) & (final.RESULT_STAT_BP == state)].user_id.nunique())/(final[(final.profile_hypertensive==False) &(final.SYSTOLIC.isnull()==False) & (final.RESULT_STAT_BP != 'High')].user_id.nunique()))*100)
    
bp_series_male.append(sum(bp_series_male)) 
bp_series_female.append(sum(bp_series_female)) 
bp_series_total.append(sum(bp_series_total)) 

bp_series_male_percent.append(sum(bp_series_male_percent)) 
bp_series_female_percent.append(sum(bp_series_female_percent)) 
bp_series_total_percent.append(sum(bp_series_total_percent)) 

    
    
bp_table_before.loc['পুরুষ'] = [bangla.convert_english_digit_to_bangla_digit(v) for v in bp_series_male]
bp_table_before.loc['পুরুষ(%)'] = [bangla.convert_english_digit_to_bangla_digit(v) for v in bp_series_male_percent]
bp_table_before.loc['মহিলা'] = [bangla.convert_english_digit_to_bangla_digit(v) for v in bp_series_female]
bp_table_before.loc['মহিলা(%)'] = [bangla.convert_english_digit_to_bangla_digit(v) for v in bp_series_female_percent]
bp_table_before.loc['মোট'] =  [bangla.convert_english_digit_to_bangla_digit(v) for v in bp_series_total]
bp_table_before.loc['মোট(%)'] = [bangla.convert_english_digit_to_bangla_digit(v) for v in bp_series_total_percent]


bp_table_before

,নিম্ন রক্তচাপ,স্বাভাবিক,উচ্চ রক্তচাপ (পূর্বাবস্থা),উচ্চ রক্তচাপ (Mild High),উচ্চ রক্তচাপ (Moderate High),মাত্রাতিরিক্ত রক্তচাপ,সর্বমোট
লিঙ্গ,,,,,,,
পুরুষ,২১৭০,৮৬০৫,৭৬২১,৩২৯৭,১৩৬৯,৪৭১,২৩৫৩৩
পুরুষ(%),২.১১৩২৫৮৯৯৫৯৫৮৫১৪,৮.৩৭৯৯৯৭০৭৮৪৪৩৭৮৪,৭.৪২১৭২৬৬৩৯৭২৩৪২৭,৩.২১০৭৯০২৮০৯৫৬৩২৩,১.৩৩৩২০৩৪৮৬৩৯০৪১৭৪,০.৪৫৮৬৮৪৩২৫৮৫০৯০৩২,২২.৯১৭৬৬০৮০৭৩২৩৩৭
মহিলা,১১৪৫০,৩৫৮০৮,২০৩৩০,৭৪৭৬,২৯০১,১০৩৪,৭৮৯৯৯
মহিলা(%),১১.১৫০৬০৬২২২৯১৪৭৪,৩৪.৮৭১৬৯৪৯৮৯৫৩১০৮৬,১৯.৭৯৮৪১২৬২১১২২৮৫,৭.২৮০৫১৮০৮৯৩০২২৩৫,২.৮২৫১৪৪৮৬০৪৯৫৬৯০৬,১.০০৬৯৬৩০৪২৩১৩৮৭২৫,৭৬.৯৩৩৩৩৯৮২৫৬৮০৪৮
মোট,১৩৬৩০,৪৪৪৭২,২৮০০৬,১০৭৯২,৪২৭৯,১৫০৬,১০২৬৮৫
মোট(%),১৩.২৭৩৬০৩৭৩৯৫৯১৯৫৫,৪৩.৩০৯১৪৯৩৪০২১৫২২,২৭.২৭৩৭০১১২৪৭৯৯১৪৩,১০.৫০৯৮১১৫৫৯৬২৪০৯৪,৪.১৬৭১১৩০১৫৫৩২৯৪,১.৪৬৬৬২১২২০২৩৬৬৪৬,১০০.০


In [7]:

#for blood pressure
bp_table_after= pd.DataFrame(columns=['নিম্ন রক্তচাপ', 'স্বাভাবিক', 'উচ্চ রক্তচাপ (Mild High)', 'উচ্চ রক্তচাপ (Moderate High)', 'মাত্রাতিরিক্ত রক্তচাপ','সর্বমোট'])
bp_table_after.index.name ='লিঙ্গ'

bp_series_male = []
bp_series_male_percent = []
bp_series_female = []
bp_series_female_percent = []
bp_series_total = []
bp_series_total_percent = []

for state in BP_col_After:
    bp_series_male.append(final[(final.profile_hypertensive==True) & (final.SYSTOLIC.isnull() == False) & (final.gender=='Male') & (final.RESULT_STAT_BP == state)].user_id.nunique())
    bp_series_male_percent.append(((final[(final.profile_hypertensive==True) & (final.SYSTOLIC.isnull() == False) & (final.gender=='Male') & (final.RESULT_STAT_BP == state)].user_id.nunique())/(final[(final.profile_hypertensive==True) &(final.SYSTOLIC.isnull()==False) & (final.RESULT_STAT_BP != 'Prehypertension') & (final.RESULT_STAT_BP != 'High')].user_id.nunique()))*100)
    bp_series_female.append(final[(final.profile_hypertensive==True) & (final.SYSTOLIC.isnull() == False) & (final.gender=='Female') & (final.RESULT_STAT_BP == state)].user_id.nunique())
    bp_series_female_percent.append(((final[(final.profile_hypertensive==True) & (final.SYSTOLIC.isnull() == False) & (final.gender=='Female') & (final.RESULT_STAT_BP == state)].user_id.nunique())/(final[(final.profile_hypertensive==True) &(final.SYSTOLIC.isnull()==False) & (final.RESULT_STAT_BP != 'High') & (final.RESULT_STAT_BP != 'Prehypertension')].user_id.nunique()))*100)
    bp_series_total.append(final[(final.profile_hypertensive==True) &(final.SYSTOLIC.isnull()==False) & (final.RESULT_STAT_BP == state)].user_id.nunique())
    bp_series_total_percent.append(((final[(final.profile_hypertensive==True) & (final.SYSTOLIC.isnull() == False) & (final.RESULT_STAT_BP == state)].user_id.nunique())/(final[(final.profile_hypertensive==True) &(final.SYSTOLIC.isnull()==False) & (final.RESULT_STAT_BP != 'High') & (final.RESULT_STAT_BP != 'Prehypertension')].user_id.nunique()))*100)
    
bp_series_male.append(sum(bp_series_male)) 
bp_series_female.append(sum(bp_series_female)) 
bp_series_total.append(sum(bp_series_total)) 

bp_series_male_percent.append(sum(bp_series_male_percent)) 
bp_series_female_percent.append(sum(bp_series_female_percent)) 
bp_series_total_percent.append(sum(bp_series_total_percent)) 

    
    
bp_table_after.loc['পুরুষ'] = [bangla.convert_english_digit_to_bangla_digit(v) for v in bp_series_male] 
bp_table_after.loc['পুরুষ(%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in bp_series_male_percent]  
bp_table_after.loc['মহিলা'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in bp_series_female]  
bp_table_after.loc['মহিলা(%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in bp_series_female_percent]  
bp_table_after.loc['মোট'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in bp_series_total]  
bp_table_after.loc['মোট(%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in bp_series_total_percent]  


bp_table_after


,নিম্ন রক্তচাপ,স্বাভাবিক,উচ্চ রক্তচাপ (Mild High),উচ্চ রক্তচাপ (Moderate High),মাত্রাতিরিক্ত রক্তচাপ,সর্বমোট
লিঙ্গ,,,,,,
পুরুষ,৮৮,৯৪২,৪৪১,২০৮,৭৪,১৭৫৩
পুরুষ(%),১.০৪৩৩৯৫৭৭৮৯৮৯৮০৩২,১১.১৬৯০৭৭৫৪৩২৭৭২১১,৫.২২৮৮৩৫৬৬৫১৬৪৮০৯,২.৪৬৬২০৮২০৪৮৮৪৯৮৯৪,০.৮৭৭৪০০৯৯৫৯৬৮৬৯৮১,২০.৭৮৪৯১৮১৮৮২৮৫৫১৩
মহিলা,৪৮৭,৩৮৫৬,১৫০৪,৬৩৭,১৯৪,৬৬৭৮
মহিলা(%),৫.৭৭৪২৪৭০৯৫০৯১২৯৭,৪৫.৭১৯৭০৫৯৫২০৯৮৬৫,১৭.৮৩২৫৮২৪০৪৫৫২৯৯৮,৭.৫৫২৭৬২৬২৭৪৬০২৮,২.৩০০২১৩৪২১৮৬৩৮৮৪৬,৭৯.১৭৯৫১১৫০১০৬৭১১
মোট,৫৭৫,৪৭৯৯,১৯৪৭,৮৪৫,২৬৮,৮৪৩৪
মোট(%),৬.৮১৭৬৪২৮৭৪০৮১১০১,৫৬.৯০০৬৪০২৬৫৫৯১৬৫,২৩.০৮৫১৩১৬১০১৪৯৩৯৬,১০.০১৮৯৭০৮৩২৩৪৫২৭,৩.১৭৭৬১৪৪১৭৮৩২৫৮২৫,১০০.০


# Diabetic

In [8]:
#for blood pressure
dia_table_before = pd.DataFrame(columns=['কম ( হাইপোগ্লাইসেমিয়া)', 'স্বাভাবিক', 'ডায়াবেটিস (পূর্বাবস্থা)', 'ডায়াবেটিস (নিশ্চিত হোন)','সর্বমোট'])
dia_table_before.index.name = 'লিঙ্গ'

dia_series_male = []
dia_series_male_percent = []
dia_series_female = []
dia_series_female_percent = []
dia_series_total = []
dia_series_total_percent = []

for state in SUGAR_col_before:
    dia_series_male.append(final[(final.diabetic == False) & (final.SUGAR > 0) & (final.RESULT_STAT_SUGAR == state) & (final.gender == 'Male')].user_id.nunique())
    dia_series_male_percent.append(((final[(final.diabetic == False) & (final.SUGAR > 0) & (final.RESULT_STAT_SUGAR == state) & (final.gender == 'Male')].user_id.nunique()) / (final[(final.diabetic==False) &(final.SUGAR > 0) & (final.RESULT_STAT_SUGAR != 'High') & (final.RESULT_STAT_SUGAR != 'Low') & (final.RESULT_STAT_SUGAR != 'HIGH (Borderline)')].user_id.nunique()))*100)
    dia_series_female.append(final[(final.diabetic == False) & (final.SUGAR > 0) & (final.RESULT_STAT_SUGAR == state) & (final.gender == 'Female')].user_id.nunique())
    dia_series_female_percent.append(((final[(final.diabetic == False) & (final.SUGAR > 0) & (final.RESULT_STAT_SUGAR == state) & (final.gender == 'Female')].user_id.nunique()) / (final[(final.diabetic==False) &(final.SUGAR > 0) & (final.RESULT_STAT_SUGAR != 'High') & (final.RESULT_STAT_SUGAR != 'Low') & (final.RESULT_STAT_SUGAR != 'HIGH (Borderline)')].user_id.nunique()))*100)
    dia_series_total.append(final[(final.diabetic == False) & (final.SUGAR > 0) & (final.RESULT_STAT_SUGAR == state)].user_id.nunique())
    dia_series_total_percent.append(((final[(final.diabetic == False) & (final.SUGAR > 0) & (final.RESULT_STAT_SUGAR == state)].user_id.nunique()) / (final[(final.diabetic==False) &(final.SUGAR > 0) & (final.RESULT_STAT_SUGAR != 'High') & (final.RESULT_STAT_SUGAR != 'Low') & (final.RESULT_STAT_SUGAR != 'HIGH (Borderline)')].user_id.nunique()))*100)
    
dia_series_male.append(sum(dia_series_male)) 
dia_series_female.append(sum(dia_series_female)) 
dia_series_total.append(sum(dia_series_total)) 

dia_series_male_percent.append(sum(dia_series_male_percent)) 
dia_series_female_percent.append(sum(dia_series_female_percent)) 
dia_series_total_percent.append(sum(dia_series_total_percent)) 

    
    
dia_table_before.loc['পুরুষ'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in dia_series_male]   
dia_table_before.loc['পুরুষ(%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in dia_series_male_percent]   
dia_table_before.loc['মহিলা'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in dia_series_female]   
dia_table_before.loc['মহিলা(%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in dia_series_female_percent]   
dia_table_before.loc['মোট'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in dia_series_total]   
dia_table_before.loc['মোট(%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in dia_series_total_percent]   


dia_table_before

,কম ( হাইপোগ্লাইসেমিয়া),স্বাভাবিক,ডায়াবেটিস (পূর্বাবস্থা),ডায়াবেটিস (নিশ্চিত হোন),সর্বমোট
লিঙ্গ,,,,,
পুরুষ,১৪,৭৭২,১৪৫,৬৯০,১৬২১
পুরুষ(%),০.২৩৭০০৬৯৪০৯১৭৫৫৫৪৫,১৩.০৬৯২৩৯৮৮৪৮৮২৩৪৫,২.৪৫৪৭১৪৭৪৫২১৭৫৩৮৪,১১.৬৮১০৫৬৩৭৩৭৯৩৮০৩,২৭.৪৪২০১৭৯৪৪৮১১২৪
মহিলা,২৮,২২২৯,৩৮৩,১৬২৮,৪২৬৮
মহিলা(%),০.৪৭৪০১৩৮৮১৮৩৫১১০৯,৩৭.৭৩৪৮৯০৮০৭৫১৬৫১,৬.৪৮৩৮৩২৭৪০৮১৫৯৮১৫,২৭.৫৬০৫২১৪১৫২৭০০১৭,৭২.২৫৩২৫৮৮৪৫৪৩৭৬১
মোট,৪২,৩০১১,৫২৯,২৩২৫,৫৯০৭
মোট(%),০.৭১১০২০৮২২৭৫২৬৬৬৩,৫০.৯৭৩৪২১৩৬৪৪৮২৮২,৮.৯৫৫৪৭৬৫৫৩২৪১৯১৫,৩৯.৩৬০০৮১২৫৯৫২২৬,১০০.০


In [9]:
#for blood pressure
dia_table_after = pd.DataFrame(columns=['কম ( হাইপোগ্লাইসেমিয়া)', 'স্বাভাবিক', 'বেশি', 'মাত্রাতিরিক্ত','সর্বমোট'])
dia_table_after.index.name = 'লিঙ্গ'

dia_series_male = []
dia_series_male_percent = []
dia_series_female = []
dia_series_female_percent = []
dia_series_total = []
dia_series_total_percent = []

for state in SUGAR_col_after:
    dia_series_male.append(final[(final.diabetic == True) & (final.SUGAR > 0) & (final.RESULT_STAT_SUGAR == state) & (final.gender == 'Male')].user_id.nunique())
    dia_series_male_percent.append(((final[(final.diabetic == True) & (final.SUGAR > 0) & (final.RESULT_STAT_SUGAR == state) & (final.gender == 'Male')].user_id.nunique()) / (final[(final.diabetic==True) &(final.SUGAR > 0) & (final.RESULT_STAT_SUGAR != 'PRE- DIABETIC') & (final.RESULT_STAT_SUGAR != 'Low') & (final.RESULT_STAT_SUGAR != 'DIABETIC (need confirmation)')].user_id.nunique()))*100)
    dia_series_female.append(final[(final.diabetic == True) & (final.SUGAR > 0) & (final.RESULT_STAT_SUGAR == state) & (final.gender == 'Female')].user_id.nunique())
    dia_series_female_percent.append(((final[(final.diabetic == True) & (final.SUGAR > 0) & (final.RESULT_STAT_SUGAR == state) & (final.gender == 'Female')].user_id.nunique()) / (final[(final.diabetic==True) &(final.SUGAR > 0) & (final.RESULT_STAT_SUGAR != 'PRE- DIABETIC') & (final.RESULT_STAT_SUGAR != 'Low') & (final.RESULT_STAT_SUGAR != 'DIABETIC (need confirmation)')].user_id.nunique()))*100)
    dia_series_total.append(final[(final.diabetic == True) & (final.SUGAR > 0) & (final.RESULT_STAT_SUGAR == state)].user_id.nunique())
    dia_series_total_percent.append(((final[(final.diabetic == True) & (final.SUGAR > 0) & (final.RESULT_STAT_SUGAR == state)].user_id.nunique()) / (final[(final.diabetic==True) &(final.SUGAR > 0) & (final.RESULT_STAT_SUGAR != 'PRE- DIABETIC') & (final.RESULT_STAT_SUGAR != 'Low') & (final.RESULT_STAT_SUGAR != 'DIABETIC (need confirmation)')].user_id.nunique()))*100)
    
dia_series_male.append(sum(dia_series_male)) 
dia_series_female.append(sum(dia_series_female)) 
dia_series_total.append(sum(dia_series_total)) 

dia_series_male_percent.append(sum(dia_series_male_percent)) 
dia_series_female_percent.append(sum(dia_series_female_percent)) 
dia_series_total_percent.append(sum(dia_series_total_percent)) 

    
    
dia_table_after.loc['পুরুষ'] = [bangla.convert_english_digit_to_bangla_digit(v) for v in dia_series_male] 
dia_table_after.loc['পুরুষ(%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in dia_series_male_percent] 
dia_table_after.loc['মহিলা'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in dia_series_female] 
dia_table_after.loc['মহিলা(%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in dia_series_female_percent] 
dia_table_after.loc['মোট'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in dia_series_total] 
dia_table_after.loc['মোট(%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in dia_series_total_percent] 


dia_table_after

,কম ( হাইপোগ্লাইসেমিয়া),স্বাভাবিক,বেশি,মাত্রাতিরিক্ত,সর্বমোট
লিঙ্গ,,,,,
পুরুষ,১,৪০,২৮,১৬৯,২৩৮
পুরুষ(%),০.০৮৩৯৬৩০৫৬২৫৫২৪৭৬৯,৩.৩৫৮৫২২২৫০২০৯৯০৭৭,২.৩৫০৯৬৫৫৭৫১৪৬৯৩৫৪,১৪.১৮৯৭৫৬৫০৭১৩৬৮৬১,১৯.৯৮৩২০৭৩৮৮৭৪৮৯৫
মহিলা,১,৯৫,১৬১,৬৯৬,৯৫৩
মহিলা(%),০.০৮৩৯৬৩০৫৬২৫৫২৪৭৬৯,৭.৯৭৬৪৯০৩৪৪২৪৮৫৩১,১৩.৫১৮০৫২০৫৭০৯৪৮৭৯,৫৮.৪৩৮২৮৭১৫৩৬৫২৩৯,৮০.০১৬৭৯২৬১১২৫১০৫
মোট,২,১৩৫,১৮৯,৮৬৫,১১৯১
মোট(%),০.১৬৭৯২৬১১২৫১০৪৯৫৩৮,১১.৩৩৫০১২৫৯৪৪৫৮৪৩৭,১৫.৮৬৯০১৭৬৩২২৪১৮১৪,৭২.৬২৮০৪৩৬৬০৭৮৯২৬,১০০.০


# Pulse Rate

In [10]:
#for blood pressure
pr_table = pd.DataFrame(columns=['কম', 'স্বাভাবিক', 'বেশি', 'সর্বমোট'])
pr_table.index.name = 'লিঙ্গ'

pr_series_male = []
pr_series_male_percent = []
pr_series_female = []
pr_series_female_percent = []
pr_series_total = []
pr_series_total_percent = []

for state in PR_col:
    pr_series_male.append(m_final[(m_final.PULSE_RATE.isnull() == False) & (m_final.RESULT_STAT_PR == state) & (m_final.gender == 'Male')].user_id.nunique())
    pr_series_male_percent.append(((m_final[(m_final.PULSE_RATE.isnull() == False) & (m_final.RESULT_STAT_PR == state) & (m_final.gender == 'Male')].user_id.nunique()) / (m_final[(m_final.PULSE_RATE.isnull() ==False) & ((m_final.gender == 'Male') | (m_final.gender == 'Female'))].user_id.nunique()))*100)
    pr_series_female.append(m_final[(m_final.PULSE_RATE.isnull() == False) & (m_final.RESULT_STAT_PR == state) & (m_final.gender == 'Female')].user_id.nunique())
    pr_series_female_percent.append(((m_final[(m_final.PULSE_RATE.isnull() == False) & (m_final.RESULT_STAT_PR == state) & (m_final.gender == 'Female')].user_id.nunique()) / (m_final[(m_final.PULSE_RATE.isnull() ==False) & ((m_final.gender == 'Male') | (m_final.gender == 'Female'))].user_id.nunique()))*100)
    pr_series_total.append(m_final[(m_final.PULSE_RATE.isnull() == False) & (m_final.RESULT_STAT_PR == state) & ((m_final.gender == 'Male') | (m_final.gender == 'Female'))].user_id.nunique())
    pr_series_total_percent.append(((m_final[(m_final.PULSE_RATE.isnull() == False) & (m_final.RESULT_STAT_PR == state) & ((m_final.gender == 'Male') | (m_final.gender == 'Female')) ].user_id.nunique()) / (m_final[(m_final.PULSE_RATE.isnull() ==False) & ((m_final.gender == 'Male') | (m_final.gender == 'Female'))].user_id.nunique()))*100)
    
pr_series_male.append(sum(pr_series_male)) 
pr_series_female.append(sum(pr_series_female)) 
pr_series_total.append(sum(pr_series_total)) 

pr_series_male_percent.append(sum(pr_series_male_percent)) 
pr_series_female_percent.append(sum(pr_series_female_percent)) 
pr_series_total_percent.append(sum(pr_series_total_percent)) 

    
    
pr_table.loc['পুরুষ'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in pr_series_male]  
pr_table.loc['পুরুষ(%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in pr_series_male_percent]  
pr_table.loc['মহিলা'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in pr_series_female]  
pr_table.loc['মহিলা(%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in pr_series_female_percent]  
pr_table.loc['মোট'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in pr_series_total]  
pr_table.loc['মোট(%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in pr_series_total_percent]  


pr_table

,কম,স্বাভাবিক,বেশি,সর্বমোট
লিঙ্গ,,,,
পুরুষ,১৪৮৭,২৩৫১৮,১৭৫৯,২৬৭৬৪
পুরুষ(%),১.২৭৪৪৯১৩১৭৬৮৮৬০২৬,২০.১৫৭০১৮৭০১৬৮১৬০৮,১.৫০৭৬১৯৫২১০৫৮৬৭৬৫,২২.৯৩৯১২৯৫৪০৪২৮৮৮৭
মহিলা,২১৭৩,৭৯৪৫৮,৮২৭৯,৮৯৯১০
মহিলা(%),১.৮৬২৪৫৪৩৬০০১১৬৫৬৫,৬৮.১০২৫৭৬৪০৯৪৮২৮২,৭.০৯৫৮৩৯৬৯০০৭৬৬২৩,৭৭.০৬০৮৭০৪৫৯৫৭১১
মোট,৩৬৬০,১০২৯৭৬,১০০৩৮,১১৬৬৭৪
মোট(%),৩.১৩৬৯৪৫৬৭৭৭০০২৫৯৩,৮৮.২৫৯৫৯৫১১১১৬৪৪৪,৮.৬০৩৪৫৯২১১১৩৫৩,১০০.০


# SPO2

In [11]:
#for blood pressure
spo2_table = pd.DataFrame(columns=['মাত্রাতিরিক্ত', 'কম', 'স্বাভাবিক', 'সর্বমোট'])
spo2_table.index.name = 'লিঙ্গ'

spo2_series_male = []
spo2_series_male_percent = []
spo2_series_female = []
spo2_series_female_percent = []
spo2_series_total = []
spo2_series_total_percent = []

for state in SPO2_col:
    spo2_series_male.append(m_final[(m_final.SPO2 > 0) & (m_final.RESULT_STAT_SPO2 == state) & (m_final.gender == 'Male')].user_id.nunique())
    spo2_series_male_percent.append(((m_final[(m_final.SPO2 > 0) & (m_final.RESULT_STAT_SPO2 == state) & (m_final.gender == 'Male')].user_id.nunique()) / (m_final[(m_final.SPO2 > 0)].user_id.nunique()))*100)
    spo2_series_female.append(m_final[(m_final.SPO2 > 0) & (m_final.RESULT_STAT_SPO2 == state) & (m_final.gender == 'Female')].user_id.nunique())
    spo2_series_female_percent.append(((m_final[(m_final.SPO2 > 0) & (m_final.RESULT_STAT_SPO2 == state) & (m_final.gender == 'Female')].user_id.nunique()) / (m_final[(m_final.SPO2 > 0) ].user_id.nunique()))*100)
    spo2_series_total.append(m_final[(m_final.SPO2 > 0) & (m_final.RESULT_STAT_SPO2 == state) & ((m_final.gender == 'Male') | (m_final.gender == 'Female'))].user_id.nunique())
    spo2_series_total_percent.append(((m_final[(m_final.SPO2 > 0) & (m_final.RESULT_STAT_SPO2 == state) ].user_id.nunique()) / (m_final[(m_final.SPO2 > 0)].user_id.nunique()))*100)
      
spo2_series_male.append(sum(spo2_series_male)) 
spo2_series_female.append(sum(spo2_series_female)) 
spo2_series_total.append(sum(spo2_series_total)) 

spo2_series_male_percent.append(sum(spo2_series_male_percent)) 
spo2_series_female_percent.append(sum(spo2_series_female_percent)) 
spo2_series_total_percent.append(sum(spo2_series_total_percent)) 

    
    
spo2_table.loc['পুরুষ'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in spo2_series_male] 
spo2_table.loc['পুরুষ(%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in spo2_series_male_percent] 
spo2_table.loc['মহিলা'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in spo2_series_female] 
spo2_table.loc['মহিলা(%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in spo2_series_female_percent] 
spo2_table.loc['মোট'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in spo2_series_total] 
spo2_table.loc['মোট(%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in spo2_series_total_percent] 


spo2_table

,মাত্রাতিরিক্ত,কম,স্বাভাবিক,সর্বমোট
লিঙ্গ,,,,
পুরুষ,২৯১,৩৩৯,১১৪১০,১২০৪০
পুরুষ(%),০.৮২৩৯১৯১৩৭০০৮৪০৯,০.৯৫৯৮২৩৩২৪৫৫৬১৮৭৯,৩২.৩০৫৫৫৭৯১৫০০৩২৫৬,৩৪.০৮৯৩০০৩৭৬৫৬৭৮৫৬
মহিলা,৪২৯,৫৩৭,২২৩১৩,২৩২৭৯
মহিলা(%),১.২১৪৬৪৩৬৭৬২০৮২৭৩৩,১.৫২০৪২৮০৯৮১৯০৭৭৫৬,৬৩.১৭৫৬২৭৮৪৯০৩৩১,৬৫.৯১০৬৯৯৬২৩৪৩২১৫
মোট,৭২০,৮৭৬,৩৩৭২৩,৩৫৩১৯
মোট(%),২.০৩৮৫৬২৮১৩২১৬৬৮২,২.৪৮০২৫১৪২২৭৪৬৯৬৩৬,৯৫.৪৮১১৮৫৭৬৪০৩৬৩৫,১০০.০


# BMI


In [12]:
names=[u'', u'', u'লিঙ্গ']

In [13]:
BMI_table = pd.DataFrame(columns=pd.MultiIndex.from_product([['কম ওজন', 'স্বাভাবিক ওজন', 'বেশি ওজন', 'স্থুলতা', 'সর্বমোট'], ['শিশু', 'কিশোর / কিশোরী']], names=[ u'', u'লিঙ্গ']))



TK=[]
male=[]
female=[]
male_percent=[]
female_percent=[]
Mkishor = []
Fkishor = []
Mshishu = []
Fshishu = []
total_row = []
total_row_percent = []
total_shishu = []
total_kishor = []
for state in BMI_col_un18:
    
    K=[]
    TS=[]
    TPS = []
    
    
    
    for g in gen_col:    
        S = []
        P = []
        S.append(m_final[(m_final.BMI > 0) & (m_final.RESULT_STAT_BMI == state) & (m_final.age >= 0) & (m_final.age < 6) & (m_final.gender == g)].user_id.nunique())
        P.append(((m_final[(m_final.BMI > 0) & (m_final.RESULT_STAT_BMI == state) & (m_final.age >= 0) & (m_final.age < 6) & (m_final.gender == g)].user_id.nunique())/ (m_final[(m_final.BMI>0) & (m_final.age>=0) & (m_final.age<6)& (m_final.RESULT_STAT_BMI != 'Highly Obesity') & (m_final.RESULT_STAT_BMI != 'Morbid Obesity')].user_id.nunique()))*100)
           
        
        S.append(m_final[(m_final.BMI > 0) & (m_final.RESULT_STAT_BMI == state) & (m_final.age >= 6) & (m_final.age < 18) & (m_final.gender == g)].user_id.nunique())
        P.append(((m_final[(m_final.BMI > 0) & (m_final.RESULT_STAT_BMI == state) & (m_final.age >= 6) & (m_final.age < 18) & (m_final.gender == g)].user_id.nunique())/ (m_final[(m_final.BMI>0) & (m_final.age>=6) & (m_final.age<18)& (m_final.RESULT_STAT_BMI != 'Highly Obesity') & (m_final.RESULT_STAT_BMI != 'Morbid Obesity')].user_id.nunique()))*100)
         
        if g == 'Male':
            male.append(S)
            male_percent.append(P)
            Mshishu.append(m_final[(m_final.BMI > 0) & (m_final.RESULT_STAT_BMI == state) & (m_final.age >= 0) & (m_final.age < 6) & (m_final.gender == g)].user_id.nunique())
            Mkishor.append(m_final[(m_final.BMI > 0) & (m_final.RESULT_STAT_BMI == state) & (m_final.age >= 6) & (m_final.age < 18) & (m_final.gender == g)].user_id.nunique())
       
        if g == 'Female':
            female.append(S)
            female_percent.append(P)
            Fshishu.append(m_final[(m_final.BMI > 0) & (m_final.RESULT_STAT_BMI == state) & (m_final.age >= 0) & (m_final.age < 6) & (m_final.gender == g)].user_id.nunique())
            Fkishor.append(m_final[(m_final.BMI > 0) & (m_final.RESULT_STAT_BMI == state) & (m_final.age >= 6) & (m_final.age < 18) & (m_final.gender == g)].user_id.nunique())
       
    TS.append(m_final[(m_final.BMI > 0) & (m_final.RESULT_STAT_BMI == state) & (m_final.age >= 0) & (m_final.age < 6)].user_id.nunique())
    TPS.append(((m_final[(m_final.BMI > 0) & (m_final.RESULT_STAT_BMI == state) & (m_final.age >= 0) & (m_final.age < 6)].user_id.nunique())/(m_final[(m_final.BMI>0)&(m_final.age>=0)&(m_final.age<6)& (m_final.RESULT_STAT_BMI != 'Highly Obesity') & (m_final.RESULT_STAT_BMI != 'Morbid Obesity')].user_id.nunique()))*100)
    
    total_shishu.append(m_final[(m_final.BMI > 0) & (m_final.RESULT_STAT_BMI == state) & (m_final.age >= 0) & (m_final.age < 6)].user_id.nunique())
        
    TS.append(m_final[(m_final.BMI > 0) & (m_final.RESULT_STAT_BMI == state) & (m_final.age >= 6) & (m_final.age < 18)].user_id.nunique())
    TPS.append(((m_final[(m_final.BMI > 0) & (m_final.RESULT_STAT_BMI == state) & (m_final.age >= 6) & (m_final.age < 18)].user_id.nunique())/(m_final[(m_final.BMI>0)&(m_final.age>=6)&(m_final.age<18)& (m_final.RESULT_STAT_BMI != 'Highly Obesity') & (m_final.RESULT_STAT_BMI != 'Morbid Obesity')].user_id.nunique()))*100)
    
    total_kishor.append(m_final[(m_final.BMI > 0) & (m_final.RESULT_STAT_BMI == state) & (m_final.age >= 6) & (m_final.age < 18)].user_id.nunique())
    total_row.append(TS)
    total_row_percent.append(TPS)
    TS=[]
    
    #shishu = list(filter(None, shishu))
    #kishor = list(filter(None, kishor)) 
total = []
total.append(sum(Mshishu))
total.append(sum(Mkishor))
male.append(total)  

total = []
total.append((sum(Mshishu)/m_final[((m_final.BMI>0) & (m_final.age>=0) & (m_final.age<6)& (m_final.RESULT_STAT_BMI != 'Highly Obesity') & (m_final.RESULT_STAT_BMI != 'Morbid Obesity'))].user_id.nunique())*100)
total.append((sum(Mkishor)/m_final[((m_final.BMI>0) & (m_final.age>=6) & (m_final.age<18)& (m_final.RESULT_STAT_BMI != 'Highly Obesity') & (m_final.RESULT_STAT_BMI != 'Morbid Obesity'))].user_id.nunique())*100)
male_percent.append(total)  

total = []
total.append(sum(Fshishu))
total.append(sum(Fkishor))
female.append(total)

total = []
total.append((sum(Fshishu)/m_final[((m_final.BMI>0) & (m_final.age>=0) & (m_final.age<6)& (m_final.RESULT_STAT_BMI != 'Highly Obesity') & (m_final.RESULT_STAT_BMI != 'Morbid Obesity'))].user_id.nunique())*100)
total.append((sum(Fkishor)/m_final[((m_final.BMI>0) & (m_final.age>=6) & (m_final.age<18)& (m_final.RESULT_STAT_BMI != 'Highly Obesity') & (m_final.RESULT_STAT_BMI != 'Morbid Obesity'))].user_id.nunique())*100)
female_percent.append(total)  

total = []
total.append(sum(total_shishu))
total.append(sum(total_kishor))
total_row.append(total)

total = []
total.append((sum(total_shishu)/m_final[((m_final.BMI>0) & (m_final.age>=0) & (m_final.age<6)& (m_final.RESULT_STAT_BMI != 'Highly Obesity') & (m_final.RESULT_STAT_BMI != 'Morbid Obesity'))].user_id.nunique())*100)
total.append((sum(total_kishor)/m_final[((m_final.BMI>0) & (m_final.age>=6) & (m_final.age<18)& (m_final.RESULT_STAT_BMI != 'Highly Obesity') & (m_final.RESULT_STAT_BMI != 'Morbid Obesity'))].user_id.nunique())*100)
total_row_percent.append(total)

#print(total_row)
        
#total_shishu = list(filter(None, total_shishu))
#total_kishor = list(filter(None, total_kishor))

#TS.append(sum(total_shishu))
#TS.append(sum(total_kishor))

BMI_table.loc['পুরুষ'] = [bangla.convert_english_digit_to_bangla_digit(v) for v in list(itertools.chain.from_iterable(male))]  
BMI_table.loc['পুরুষ (%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in list(itertools.chain.from_iterable(male_percent))]   

BMI_table.loc['মহিলা'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in list(itertools.chain.from_iterable(female))]  
BMI_table.loc['মহিলা (%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in list(itertools.chain.from_iterable(female_percent))]  

BMI_table.loc['মোট'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in list(itertools.chain.from_iterable(total_row))]  
BMI_table.loc['মোট (%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in list(itertools.chain.from_iterable(total_row_percent))]  
#male
BMI_table
#sum(total_kishor)
#m_final[((m_final.BMI>0) & (m_final.age>=6) & (m_final.age<18)) & (m_final.RESULT_STAT_BMI != 'Highly Obesity') & (m_final.RESULT_STAT_BMI != 'Morbid Obesity')].user_id.nunique()

কম ওজন                           স্বাভাবিক ওজন  \
লিঙ্গ                    শিশু      কিশোর / কিশোরী                শিশু   
পুরুষ                     ১৮৬                 ৩৪৯                 ৩৮৮   
পুরুষ (%)  ১১.৪৯৫৬৭৩৬৭১১৯৯০১১   ৬.৭৪৭৮৭৩১৬৩১৮৬৩৮৮  ২৩.৯৮০২২২৪৯৬৯০৯৭৬৪   
মহিলা                     ১৮০                 ৩৭১                 ৩৬৬   
মহিলা (%)  ১১.১২৪৮৪৫৪৮৮২৫৭১০৮   ৭.১৭৩২৪০৫২৫৯০৮৭৩৯   ২২.৬২০৫১৯১৫৯৪৫৬১২   
মোট                       ৩৬৬                 ৭২০                 ৭৫৫   
মোট (%)     ২২.৬২০৫১৯১৫৯৪৫৬১২  ১৩.৯২১১১৩৬৮৯০৯৫১২৮   ৪৬.৬৬২৫৪৬৩৫৩৫২২৮৬   

                                        বেশি ওজন                      \
লিঙ্গ          কিশোর / কিশোরী               শিশু      কিশোর / কিশোরী   
পুরুষ                    ১২৯১                 ৪৮                 ২৩৮   
পুরুষ (%)  ২৪.৯৬১৩৩০২৩৯৭৫২৫১৩  ২.৯৬৬৬২৫৪৬৩৫৩৫২২৯    ৪.৬০১৭০১৪৬৯৪৫০৮৯   
মহিলা                    ১৯৮৫                 ৫১                 ৩১২   
মহিলা (%)   ৩৮.৩৭৯৭৩৭০৪৫৬৩০৩২   ৩.১৫২০৩৯৫৫৫০০৬১৮  ৬.০৩২৪৮২৫৯৮৬০৭৮৮৮৫   
মোট                      ৩২৭৭                 ৯৯                 ৫৫০   
মোট (%)     ৬৩.৩৬০৪০২১৬৫৫০৬৫৭  ৬.১১৮৬৬৫০১৮৫৪১৪০৯  ১০.৬৩৪১৮৪০৬৮০৫৮৭৭৮   

                      স্থুলতা                                সর্বমোট  \
লিঙ্গ                    শিশু      কিশোর / কিশোরী               শিশু   
পুরুষ                     ২২০                 ৩১৭                ৮৪২   
পুরুষ (%)  ১৩.৫৯৭০৩৩৩৭৪৫৩৬৪৬৬  ৬.১২৯১৫৬৯৯৯২২৬৬০৫৫  ৫২.০৩৯৫৫৫০০৬১৮০৪৭   
মহিলা                     ১৭৮                 ৩০৮                ৭৭৫   
মহিলা (%)   ১১.০০১২৩৬০৯৩৯৪৩১৪   ৫.৯৫৫১৪৩০৭৮১১২৯১৬  ৪৭.৮৯৮৬৪০২৯৬৬৬২৫৫   
মোট                       ৩৯৮                 ৬২৫               ১৬১৮   
মোট (%)    ২৪.৫৯৮২৬৯৪৬৮৪৭৯৬০৭   ১২.০৮৪৩০০০৭৭৩৩৯৫২              ১০০.০   

                               
লিঙ্গ          কিশোর / কিশোরী  
পুরুষ                    ২১৯৫  
পুরুষ (%)   ৪২.৪৪০০৬১৮৭১৬১৬৩৯  
মহিলা                    ২৯৭৬  
মহিলা (%)  ৫৭.৫৪০৬০৩২৪৮২৫৯৮৫৬  
মোট                      ৫১৭২  
মোট (%)                 ১০০.০

In [14]:
BMI_tab = pd.DataFrame(columns=pd.MultiIndex.from_product([['কম ওজন', 'স্বাভাবিক ওজন', 'বেশি ওজন', 'স্থুলতা','অনেক বেশি ওজন','মাত্রাতিরিক্ত'], ['পুরুষ', 'মহিলা','মোট']], names=[ u'', u'বয়স সীমা']))
BMI_tab["সর্বমোট"] = 0


boyosh_shima = ['young', 'middle_age', 'adult']

jubok = []
jubok_per = []
mid_age = []
mid_age_per = []
old_age = []
old_age_per = []
total_adult=[]
total_jub=[]
total_mid=[]

total_adult_per=[]
total_jub_per=[]
total_mid_per=[]


for age_range in boyosh_shima:
    
    
    
    if age_range == 'young':
        for state in BMI_col_ov18:
            y=[]
            y.append(m_final[(m_final.gender=='Male') & (m_final.age>=18) & (m_final.age<36) & (m_final.BMI>0) & (m_final.RESULT_STAT_BMI==state)].user_id.nunique())
            y.append(m_final[(m_final.gender=='Female') & (m_final.age>=18) & (m_final.age<36) & (m_final.BMI>0) & (m_final.RESULT_STAT_BMI==state)].user_id.nunique())
            total_jub.append(sum(y))
            y.append(sum(y))
            jubok.append(y)
            
            y_per = []
            y_per.append(((m_final[((m_final.gender=='Male') & (m_final.age>=18) & (m_final.age<36) & (m_final.BMI>0) & (m_final.RESULT_STAT_BMI==state))].user_id.nunique()) / (m_final[(m_final.age>=18) & (m_final.age<36) & (m_final.BMI>0) & ((m_final.gender=='Male') | (m_final.gender=='Female'))].user_id.nunique()))*100)
            y_per.append(((m_final[(m_final.gender=='Female') & (m_final.age>=18) & (m_final.age<36) & (m_final.BMI>0) & (m_final.RESULT_STAT_BMI==state)].user_id.nunique()) / (m_final[(m_final.age>=18) & (m_final.age<36) & (m_final.BMI>0) & ((m_final.gender=='Male') | (m_final.gender=='Female'))].user_id.nunique()))*100)
            total_jub_per.append(sum(y_per))
            y_per.append(((m_final[(m_final.age>=18) & (m_final.age<36) & (m_final.BMI>0) & (m_final.RESULT_STAT_BMI==state)].user_id.nunique()) / (m_final[(m_final.age>=18) & (m_final.age<36) & (m_final.BMI>0) & ((m_final.gender=='Male') | (m_final.gender=='Male'))].user_id.nunique()))*100)
            
            jubok_per.append(y_per)
    if age_range == 'middle_age':
        for state in BMI_col_ov18:
            y=[]
            y.append(m_final[(m_final.gender=='Male') & (m_final.age>=36) & (m_final.age<60) & (m_final.BMI>0) & (m_final.RESULT_STAT_BMI==state)].user_id.nunique())
            y.append(m_final[(m_final.gender=='Female') & (m_final.age>=36) & (m_final.age<60) & (m_final.BMI>0) & (m_final.RESULT_STAT_BMI==state)].user_id.nunique())
            total_mid.append(sum(y))
            y.append(sum(y))
            mid_age.append(y)
            
            y_per = []
            y_per.append(((m_final[(m_final.gender=='Male') & (m_final.age>=36) & (m_final.age<60) & (m_final.BMI>0) & (m_final.RESULT_STAT_BMI==state)].user_id.nunique()) / (m_final[(m_final.age>=36) & (m_final.age<60) & (m_final.BMI>0) & ((m_final.gender=='Male') | (m_final.gender=='Female'))].user_id.nunique()))*100)
            y_per.append(((m_final[(m_final.gender=='Female') & (m_final.age>=36) & (m_final.age<60) & (m_final.BMI>0) & (m_final.RESULT_STAT_BMI==state)].user_id.nunique()) / (m_final[(m_final.age>=36) & (m_final.age<60) & (m_final.BMI>0) & ((m_final.gender=='Male') | (m_final.gender=='Female'))].user_id.nunique()))*100)
            total_mid_per.append(sum(y_per))
            y_per.append(((m_final[(m_final.age>=36) & (m_final.age<60) & (m_final.BMI>0) & (m_final.RESULT_STAT_BMI==state)].user_id.nunique()) / (m_final[(m_final.age>=36) & (m_final.age<60) & (m_final.BMI>0) & ((m_final.gender=='Male') | (m_final.gender=='Male'))].user_id.nunique()))*100)
           
            mid_age_per.append(y_per)
        
        
    if age_range == 'adult':
        for state in BMI_col_ov18:
            y=[]
            y.append(m_final[(m_final.gender=='Male') & (m_final.age>=60)  & (m_final.BMI>0) & (m_final.RESULT_STAT_BMI==state)].user_id.nunique())
            y.append(m_final[(m_final.gender=='Female') & (m_final.age>=60) & (m_final.BMI>0) & (m_final.RESULT_STAT_BMI==state)].user_id.nunique())
            total_adult.append(sum(y))
            y.append(sum(y))
            old_age.append(y)
            
            y_per = []
            y_per.append(((m_final[(m_final.gender=='Male') & (m_final.age>=60)  & (m_final.BMI>0) & (m_final.RESULT_STAT_BMI==state)].user_id.nunique()) / (m_final[(m_final.age>=60)  & (m_final.BMI>0) & ((m_final.gender=='Male') | (m_final.gender=='Female'))].user_id.nunique()))*100)
            y_per.append(((m_final[(m_final.gender=='Female') & (m_final.age>=60) & (m_final.BMI>0) & (m_final.RESULT_STAT_BMI==state)].user_id.nunique()) / (m_final[(m_final.age>=60)  & (m_final.BMI>0) & ((m_final.gender=='Male') | (m_final.gender=='Female')) ].user_id.nunique()))*100)
            total_adult_per.append(sum(y_per))
            y_per.append(((m_final[(m_final.age>=60) & (m_final.BMI>0) & (m_final.RESULT_STAT_BMI==state)& ((m_final.gender=='Male') | (m_final.gender=='Female'))].user_id.nunique()) / (m_final[(m_final.age>=60)  & (m_final.BMI>0) & ((m_final.gender=='Male') | (m_final.gender=='Female'))].user_id.nunique()))*100)
            
            old_age_per.append(y_per)
t = [] 
t.append(sum(total_jub))
jubok.append(t)

t = [] 
t.append(sum(total_jub_per))
jubok_per.append(t)

t = [] 
t.append(sum(total_mid))
mid_age.append(t)

t = [] 
t.append(sum(total_mid_per))
mid_age_per.append(t)

t = [] 
t.append(sum(total_adult))
old_age.append(t)

t = [] 
t.append(sum(total_adult_per))
old_age_per.append(t)

BMI_tab.loc['যুবক / যুবতী'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in list(itertools.chain.from_iterable(jubok))]   
BMI_tab.loc['যুবক / যুবতী (%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in list(itertools.chain.from_iterable(jubok_per))]   
BMI_tab.loc[ 'মধ্যবয়স্ক'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in list(itertools.chain.from_iterable(mid_age))]  
BMI_tab.loc[ 'মধ্যবয়স্ক (%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in list(itertools.chain.from_iterable(mid_age_per))]  
BMI_tab.loc['প্রবীণ'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in list(itertools.chain.from_iterable(old_age))]   
BMI_tab.loc['প্রবীণ (%)'] =[bangla.convert_english_digit_to_bangla_digit(v) for v in list(itertools.chain.from_iterable(old_age_per))]  
BMI_tab
#total_jub

কম ওজন                                          \
বয়স সীমা                       পুরুষ               মহিলা                 মোট   
যুবক / যুবতী                     ৪৬৯                ১১২১                ১৫৯০   
যুবক / যুবতী (%)  ৪.৭০৫৫২৮২৪৩২০২৫৬৮৫  ১১.২৪৭১১৫৪৮১০৮৭৫৮৮  ৬০.৯৪৯৪৬৪০১২২৫১১৫৬   
মধ্যবয়স্ক                        ১৯৭                 ৬৪২                 ৮৩৯   
মধ্যবয়স্ক (%)     ৩.০০৬২৫৬৬৭৬৩৩১৪৫১৩    ৯.৭৯৭০৩৯৫২৩৮৮২১৯  ৪৮.৭৫০৭২৬৩২১৯০৫৮৬৬   
প্রবীণ                            ৯৮                 ২৩৭                 ৩৩৫   
প্রবীণ (%)         ৬.০৭৯৪০৪৪৬৬৫০১২৪১  ১৪.৭০২২৩৩২৫০৬২০৩৪৭  ২০.৭৮১৬৩৭৭১৭১২১৫৮৭   

                       স্বাভাবিক ওজন                                          \
বয়স সীমা                       পুরুষ               মহিলা                 মোট   
যুবক / যুবতী                    ১৮৬৮                ৫১৪২                ৭০১০   
যুবক / যুবতী (%)  ১৮.৭৪১৮৪৮০৯৮৭২৫৭৯৩   ৫১.৫৯০২৪৭৮১৭৭৯৮৭৪   ২৬৮.৪৫৩২৯২৪৯৬১৭১৫   
মধ্যবয়স্ক                       ১৩৬৩                ৩৩৬১                ৪৭২৪   
মধ্যবয়স্ক (%)      ২০.৭৯৯৬৩৩৭৫৫৫৩১৮২  ৫১.২৮৯৪৮৫৭৩১৭২৫৯২৫  ২৭৪.৪৯১৫৭৪৬৬৫৮৯১৯৬   
প্রবীণ                           ৩৯৮                 ৬৭৯                ১০৭৭   
প্রবীণ (%)         ২৪.৬৮৯৮২৬৩০২৭২৯৫৩  ৪২.১২১৫৮৮০৮৯৩৩০০২৫   ৬৬.৮১১৪১৪৩৯২০৫৯৫৬   

                            বেশি ওজন                                         \
বয়স সীমা                       পুরুষ              মহিলা                 মোট   
যুবক / যুবতী                     ২১১                ৮৫৪                ১০৬৫   
যুবক / যুবতী (%)  ২.১১৬৯৮৬০৫৩৯৭৮১২৭৬   ৮.৫৬৮২৭৫৩০৮৫১৮১১  ৪০.৮৪৯৯২৩৪৩০৩২১৫৯৬   
মধ্যবয়স্ক                        ১৩১                ৬৫২                 ৭৮৩   
মধ্যবয়স্ক (%)     ১.৯৯৯০৮৪৩৮৮৮২৯৫৪৩৮  ৯.৯৪৯৬৪১৩৮৫৬২৪৯০৪  ৪৫.৪৯৬৮০৪১৮৩৬১৪১৭৪   
প্রবীণ                            ৪৭                ১১৭                 ১৬৪   
প্রবীণ (%)         ২.৯১৫৬৩২৭৫৪৩৪২৪৩২  ৭.২৫৮০৬৪৫১৬১২৯০৩৩  ১০.১৭৩৬৯৭২৭০৪৭১৪৬৫   

                              স্থুলতা                                          \
বয়স সীমা                        পুরুষ               মহিলা                 মোট   
যুবক / যুবতী                       ৫২                 ১৮৯                 ২৪১   
যুবক / যুবতী (%)   ০.৫২১৭২১৬৮১৫৪৯১১২১  ১.৮৯৬২৫৭৬৫০২৪৫৮১১৩   ৯.২২৬৬৪৬২৪৮০৮৫৭৫৯   
মধ্যবয়স্ক                          ১৯                 ১৩৯                 ১৫৮   
মধ্যবয়স্ক (%)      ০.২৮৯৯৪৩৫৩৭৩১১১৫৫২  ২.১২১১৬৫৮৭৮২২৩৭১৪৬   ৯.১৮০৭০৮৮৯০১৮০১২৮   
প্রবীণ                              ৩                  ২২                  ২৫   
প্রবীণ (%)        ০.১৮৬১০৪২১৮৩৬২২৮২৮৯  ১.৩৬৪৭৬৪২৬৭৯৯০০৭৪৪  ১.৫৫০৮৬৮৪৮৬৩৫২৩৫৭৪   

                         অনেক বেশি ওজন                       \
বয়স সীমা                         পুরুষ                মহিলা   
যুবক / যুবতী                         ৬                   ২২   
যুবক / যুবতী (%)  ০.০৬০১৯৮৬৫৫৫৬৩৩৫৯০৮৫  ০.২২০৭২৮৪০৩৭৩২৩১৬৬৭   
মধ্যবয়স্ক                            ৭                   ২২   
মধ্যবয়স্ক (%)      ০.১০৬৮২১৩০৩২১৯৮৯৯২৭  ০.৩৩৫৭২৪০৯৫৮৩৩৯৬৯১৭   
প্রবীণ                               ১                    ৫   
প্রবীণ (%)        ০.০৬২০৩৪৭৩৯৪৫৪০৯৪২৯৫   ০.৩১০১৭৩৬৯৭২৭০৪৭১৫   

                                              মাত্রাতিরিক্ত  \
বয়স সীমা                          মোট                 পুরুষ   
যুবক / যুবতী                       ২৮                     ৬   
যুবক / যুবতী (%)   ১.০৭১৯৭৫৪৯৭৭০২৯০৯৭  ০.০৬০১৯৮৬৫৫৫৬৩৩৫৯০৮৫   
মধ্যবয়স্ক                          ২৯                     ৪   
মধ্যবয়স্ক (%)        ১.৬৮৫০৬৬৮২১৬১৫৩৪    ০.০৬১০৪০৭৪৪৬৯৭০৮৫৩   
প্রবীণ                              ৬                     ০   
প্রবীণ (%)        ০.৩৭২২০৮৪৩৬৭২৪৫৬৫৭৭                   ০.০   

                                                                    সর্বমোট  
বয়স সীমা                       মহিলা                 মোট                     
যুবক / যুবতী                      ২৭                  ৩৩               ৯৯৬৭  
যুবক / যুবতী (%)  ০.২৭০৮৯৩৯৫০০৩৫১১৫৯  ১.২৬৩৩৯৯৬৯৩৭২১২৮৬৪              ১০০.০  
মধ্যবয়স্ক                         ১৬                  ২০               ৬৫৫৩  
মধ্যবয়স্ক (%)     ০.২৪৪১৬২৯৭৮৭৮৮৩৪১২  ১.১৬২১১৫০৪৯৩৮৯৮৮৯৫  ৯৯.৯৯৯৯৯

# MUAC

In [15]:
muac_table = pd.DataFrame(columns=['বেশি অপুষ্ট','কম অপুষ্ট', 'স্বাভাবিশট' ,'সর্বমোট'])
muac_table.index.name = 'লিঙ্গ'

boy = []
girl = []
total_bg = []
boy_per = []
girl_per = []
total_bg_per = []


for state in MUAC_col:
    
    boy.append(m_final[(m_final.MUAC.isnull() == False) & (m_final.age <= 5) & (m_final.RESULT_STAT_MUAC == state) & (m_final.gender == 'Male')].user_id.nunique())
    girl.append(m_final[(m_final.MUAC.isnull() == False) & (m_final.age <= 5) & (m_final.RESULT_STAT_MUAC == state) & (m_final.gender == 'Female')].user_id.nunique())
    total_bg.append(m_final[(m_final.MUAC.isnull() == False) & (m_final.age <= 5) & (m_final.RESULT_STAT_MUAC == state) & ((m_final.gender == 'Female') | (m_final.gender == 'Male'))].user_id.nunique())
    
    boy_per.append(((m_final[(m_final.MUAC.isnull() == False) & (m_final.age <= 5) & (m_final.RESULT_STAT_MUAC == state) & (m_final.gender == 'Male')].user_id.nunique()) / (m_final[(m_final.MUAC.isnull() == False) & (m_final.age <= 5) & ((m_final.gender == 'Female') | (m_final.gender == 'Male'))].user_id.nunique()))*100)
    girl_per.append(((m_final[(m_final.MUAC.isnull() == False) & (m_final.age <= 5) & (m_final.RESULT_STAT_MUAC == state) & (m_final.gender == 'Female')].user_id.nunique()) / (m_final[(m_final.MUAC.isnull() == False) & (m_final.age <= 5) & ((m_final.gender == 'Female') | (m_final.gender == 'Male'))].user_id.nunique()))*100)
    total_bg_per.append(((m_final[(m_final.MUAC.isnull() == False) & (m_final.age <= 5) & (m_final.RESULT_STAT_MUAC == state) & ((m_final.gender == 'Female') | (m_final.gender == 'Male'))].user_id.nunique()) / (m_final[(m_final.MUAC.isnull() == False) & (m_final.age <= 5) & ((m_final.gender == 'Female') | (m_final.gender == 'Male'))].user_id.nunique()))*100)

boy.append(sum(boy))    
girl.append(sum(girl)) 
total_bg.append(sum(total_bg)) 
                    
boy_per.append(sum(boy_per))
girl_per.append(sum(girl_per))
total_bg_per.append(sum(total_bg_per))  
                    
muac_table.loc["ছেলে"] =[bangla.convert_english_digit_to_bangla_digit(v) for v in boy] 
muac_table.loc["ছেলে (%)"] =[bangla.convert_english_digit_to_bangla_digit(v) for v in boy_per] 
muac_table.loc["মেয়ে"] =[bangla.convert_english_digit_to_bangla_digit(v) for v in girl] 
muac_table.loc["মেয়ে (%)"] =[bangla.convert_english_digit_to_bangla_digit(v) for v in girl_per] 
muac_table.loc["মোট"] =[bangla.convert_english_digit_to_bangla_digit(v) for v in total_bg] 
muac_table.loc["%"] =[bangla.convert_english_digit_to_bangla_digit(v) for v in total_bg_per] 
muac_table                    

,বেশি অপুষ্ট,কম অপুষ্ট,স্বাভাবিশট,সর্বমোট
লিঙ্গ,,,,
ছেলে,৩৪,৩৭৫,১৪০৬,১৮১৫
ছেলে (%),০.৯৪৬৫৪৭৮৮৪১৮৭০৮২৫,১০.৪৩৯৮৬৬৩৬৯৭১০৪৬৯,৩৯.১৪২৫৩৮৯৭৫৫০১১১,৫০.৫২৮৯৫৩২২৯৩৯৮৬৭
মেয়ে,৪৪,৩৫৬,১৩৭৭,১৭৭৭
মেয়ে (%),১.২২৪৯৪৪৩২০৭১২৬৯৪৯,৯.৯১০৯১৩১৪০৩১১৮০৪,৩৮.৩৩৫১৮৯৩০৯৫৭৬৮৪,৪৯.৪৭১০৪৬৭৭০৬০১৩৪
মোট,৭৮,৭৩১,২৭৮৩,৩৫৯২
%,২.১৭১৪৯২২০৪৮৯৯৭৭৭৩,২০.৩৫০৭৭৯৫১০০২২২৭,৭৭.৪৭৭৭২৮২৮৫০৭৭৯৪,১০০.০


In [16]:
#destination_file = pd.ExcelWriter('TosaveFiles.xlsx', engine='xlsxwriter')
#overall_table.to_excel(destination_file, sheet_name="Overall")
#bp_table_before.to_excel(destination_file, sheet_name="Blood Pressure Before")
#bp_table_after.to_excel(destination_file, sheet_name="Blood Pressure After")

#dia_table_before.to_excel(destination_file, sheet_name="Blood Glucose Before")
#dia_table_after.to_excel(destination_file, sheet_name="Blood Glucose After")

#BMI_table.to_excel(destination_file, sheet_name="BMI(Child&Teenage)")
#BMI_tab.to_excel(destination_file, sheet_name="BMI(young,mid,old)")

#pr_table.to_excel(destination_file, sheet_name="Pulse Rate")
#spo2_table.to_excel(destination_file, sheet_name="SPO2")
#muac_table.to_excel(destination_file, sheet_name="MUAC")

#destination_file.save()

In [17]:
boy_per[1]

10.439866369710469